In [1]:
import os
import sys
import datetime
import math

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas
import folium

In [2]:
datetimeFormat = "%Y-%m-%dT%H:%M:%S"
timeBucketSeconds = 60*30 #60 seconds times how many number of minutes
deResoluteFactor = 10 #10 = 1 decimal place in lat/longs  100=2 decimal places, etc
windowLength=8  #how many windows to use for a subpath.  8 windows = 2 hours
minDistance = 0.5 #minimum distance between first and last lats.  this is a hack, 0.5 is pretty good

In [3]:
class PointTime:
    def __init__(self, dt, lat, lon):
        self.dt = dt
        self.lat = float(lat)
        self.lon = float(lon)
    
    def getDT(self):
        return self.dt

    def getLat(self):
        return self.lat
    
    def getLong(self):
        return self.lon
    
    def getLatLong(self):
        return (self.lat, self.lon)
    
    def getData(self):
        return (self.dt, self.lat, self.lon)
    
    def _deResoluteFloat(self, f, p):
        x = float(int(f * p)/p)
        return x

    def getQuantizedLatLong(self, quantizeFactor):
        return (self._deResoluteFloat(self.lat, quantizeFactor), self._deResoluteFloat(self.lon, quantizeFactor))
    
    def getQuantizedLatLongStr(self, quantizeFactor):
        return str(int(self.lat * quantizeFactor)) + ":" + str(int(self.lon * quantizeFactor))
        

In [4]:
class TravellingPointTime:
    def __init__(self, name, datetimeFormat="%Y-%m-%dT%H:%M:%S"):
        self.name = name
        self._rawPointTimes = []
        self._datetimeFormat = datetimeFormat
    
    def sortPointTimes(self):
        self._rawPointTimes=sorted(self._rawPointTimes, key=lambda x: x.getDT())
        
    def getRawPointTimes(self):
        return self._rawPointTimes
    
    def addPointTime(self, pt):
        self._rawPointTimes.extend([pt])
        
    def count(self):
        return len(self._rawPointTimes)
    
    def getPath(self):
        return self._rawPointTimes
        
    def getMinDateTime(self):
        return self._rawPointTimes[0].getDT()
    
    def getMaxDateTime(self):
        return self._rawPointTimes[-1].getDT()
    
    def _interpolatePointFromNeighbors(self, dtTarget, prevPoint, nextPoint):
        #given two pointtimes and a time in between, extrapolate a lat long for that time
        dt_prev = prevPoint.getDT()
        lat_prev = float(prevPoint.getLat())
        lon_prev = float(prevPoint.getLong())
        dt_next = nextPoint.getDT()
        lat_next = float(nextPoint.getLat())
        lon_next = float(nextPoint.getLong())
        dto_prev = datetime.datetime.strptime(dt_prev, self._datetimeFormat)
        dto_next = datetime.datetime.strptime(dt_next, self._datetimeFormat)

        if(dt_prev==dt_next):
            return (lat_prev, lon_prev)

        r = (dtTarget - dto_prev) / (dto_next - dto_prev)

        tlat = ((lat_next-lat_prev) * r) + lat_prev
        tlong = ((lon_next-lon_prev) * r) + lon_prev
        return (tlat, tlong)
    
    def _getBlankPathPointsDict(self, minDT, maxDT, timeBucketSeconds):
        #returns map of all available times, pointtimes are defaulted to none
        pathpoints = {}
        dtX = minDT
        while dtX<=maxDT:
            pathpoints[dtX] = None
            dtX = dtX + datetime.timedelta(0, timeBucketSeconds)   
        return pathpoints
        
    def getQuantizedPath(self, globalMinDT, globalMaxDT, timeBucketSeconds):
        self.sortPointTimes()
        
        minDT = datetime.datetime.strptime(self.getMinDateTime(), self._datetimeFormat)
        maxDT = datetime.datetime.strptime(self.getMaxDateTime(), self._datetimeFormat)

        pathpoints = self._getBlankPathPointsDict(minDT, maxDT, timeBucketSeconds)
        
        aisShipDataLen=len(self._rawPointTimes)
        if(aisShipDataLen<2):
            return pathpoints
        
        dtX = minDT
        idxS = 1 #start with 2nd element, we always need element n-1, so we dont want to take the first element
        
        while dtX<=maxDT:
            currentPathPoint = self._rawPointTimes[idxS]
            currentPathPointDT = datetime.datetime.strptime(currentPathPoint.getDT(), self._datetimeFormat)
            
            while(idxS<aisShipDataLen) and (currentPathPointDT<dtX):
                idxS = idxS + 1
                currentPathPoint = self._rawPointTimes[idxS]
                currentPathPointDT = datetime.datetime.strptime(currentPathPoint.getDT(), self._datetimeFormat)
                
            previousPathPoint = self._rawPointTimes[idxS-1]
            previousPathPointDT = datetime.datetime.strptime(previousPathPoint.getDT(), self._datetimeFormat)
            if(previousPathPointDT<=dtX) and (currentPathPointDT>=dtX):
                (interp_lat, interp_long) = self._interpolatePointFromNeighbors(dtX, previousPathPoint, currentPathPoint)
                pathPoint = PointTime(dtX, interp_lat, interp_long)
                pathpoints[dtX] = pathPoint
            dtX = dtX + datetime.timedelta(0, timeBucketSeconds)

        return pathpoints        
    
    def total_distance(self):
        #just uses the quadratic formula, differences in lat squared, difference of lon squared, sqrrt for a rough estimate
        
        pA = self._rawPointTimes[0]
        pB = self._rawPointTimes[-1]
        (latA, lonA) = pA.getLatLong()
        (latB, lonB) = pB.getLatLong()
        dA = (latB-latA)
        dB = (lonB-lonA)
        tempDistance = math.sqrt((dA*dA) + (dB*dB))
        return tempDistance

In [5]:
sourceFile= "../../ais/AIS_2022_08_15.csv"

In [6]:
def readinSourceFile(sf):
    rv = {}
    line=None
    header=None
    c=0
    globalMinDT = None
    globalMaxDT = None
    with open(sf) as fIn:
        line=fIn.readline()
        while(line):
            lineBits = line.strip().split(",")
            lineBitsLen = len(lineBits)
            if(lineBitsLen!=17):
                print(lineBitsLen)
            if(c==0):
                header = lineBits
            else:
                mmsi = str(lineBits[0])
                if not mmsi in rv:
                    rv[mmsi] = TravellingPointTime(mmsi)
                pt = PointTime(lineBits[1], lineBits[2], lineBits[3])
                rv[mmsi].addPointTime(pt)

                if(globalMinDT is None or lineBits[1]<globalMinDT):
                    globalMinDT = lineBits[1]
                if(globalMaxDT is None or lineBits[1]>globalMaxDT):
                    globalMaxDT = lineBits[1]                    
                    
            c=c+1
            line = fIn.readline()
    return (rv, globalMinDT, globalMaxDT, c)

In [7]:
(aisData, globalMinDT, globalMaxDT, totalLines) = readinSourceFile(sourceFile)

In [8]:
minDTO = datetime.datetime.strptime(globalMinDT, datetimeFormat)
maxDTO = datetime.datetime.strptime(globalMaxDT, datetimeFormat)

In [9]:
def isUsablePath(mmsipath, minPoints=15):
    #is there movement, are there enough valid points.  good way to ignore paths with only 1-2 entries
    validPoints = 0
    firstPoint = None
    lastPoint = None
    
    for x in mmsipath:
        p = mmsipath[x]
        if(p is not None):
            validPoints = validPoints + 1
            if(firstPoint is None):
                firstPoint = p
            lastPoint = p

    if(firstPoint is not None):
        (latA, lonA) = firstPoint.getLatLong()
    if(lastPoint is not None):
        (latB, lonB) = lastPoint.getLatLong()
    
    return (firstPoint is not None) and (lastPoint is not None) and (latA!=latB) and (lonA !=lonB) and (validPoints>=minPoints)

In [25]:
c=0
subpaths = {}

for mmsi in aisData:
    pointData = aisData[mmsi]
    dis = pointData.total_distance()
    
    mmsipath = pointData.getQuantizedPath(minDTO, maxDTO, timeBucketSeconds) 
    
    if((dis<minDistance) or not isUsablePath(mmsipath)):
        continue
        
    mmsiLen = len(mmsipath)
    ks = list(mmsipath.keys())

    for subpathX in range(mmsiLen-windowLength):
        subPathStr = []
        startK = ks[subpathX]
        for subpathY in range(windowLength):
            k = ks[subpathX+subpathY]
            p = mmsipath[k].getQuantizedLatLongStr(100)
            subPathStr.extend([p])
        s = ":".join(subPathStr)
        hs = hash(s)
        
        if(hs not in subpaths):
            subpaths[hs] = {}
        if(startK not in subpaths[hs]):
            subpaths[hs][startK] = []
        subpaths[hs][startK].extend([mmsi])
    if(c%1000==0):
        print(c)
    c=c+1

0
1000
2000


In [12]:
def getLatLongtoPlot(mmsi):
    rv = []
    p1 = aisData[mmsi].getQuantizedPath(minDTO, maxDTO, timeBucketSeconds)
    for x in p1:
        (lat,lon) = p1[x].getLatLong()
        rv.extend([(lat, lon)])
    return rv

In [67]:
def mapmmsis(mmsis):
    map=folium.Map()
    colors = ['green', 'red', 'blue', 'yellow']
    colorIdx=0
    for mmsi in mmsis:
        lats = []
        lons = []    
        latlongs = getLatLongtoPlot(mmsi)
        for (lat, lon) in latlongs:
            lats.extend([lat])
            lons.extend([lon])
        df = pd.DataFrame({'longitude': lons, 'latitude': lats})

        for (lat, lon) in latlongs:
            map.add_child(folium.Marker(location=[lat,lon],icon=folium.Icon(color=colors[colorIdx])))
        colorIdx=colorIdx+1
    display(map)

In [69]:
lastMMSIStr=""
for h in subpaths:
    for k in subpaths[h]:
        mmsis = subpaths[h][k]
        if(len(mmsis)>=2):
            tempmmsisStr = str(mmsis)
            if(tempmmsisStr!=lastMMSIStr):
                lastMMSIStr=tempmmsisStr
                mapmmsis(mmsis)
            print(h, k, mmsis)
    

-6687164508542413046 2022-08-15 00:00:05 ['367543390', '367302370']
-6687164508542413046 2022-08-15 00:30:05 ['367543390', '367302370']
-6687164508542413046 2022-08-15 01:00:05 ['367543390', '367302370']
-6687164508542413046 2022-08-15 01:30:05 ['367543390', '367302370']
-6687164508542413046 2022-08-15 02:00:05 ['367543390', '367302370']
-6687164508542413046 2022-08-15 02:30:05 ['367543390', '367302370']
-6687164508542413046 2022-08-15 03:00:05 ['367543390', '367302370']


-2334879538434180632 2022-08-15 02:30:03 ['316009316', '316033903']
3666858374440535614 2022-08-15 03:00:03 ['316009316', '316033903']
507782818750665489 2022-08-15 03:30:03 ['316009316', '316033903']
-9088409850143452555 2022-08-15 04:00:03 ['316009316', '316033903']
-8782223665717912009 2022-08-15 04:30:03 ['316009316', '316033903']
8176244160059030808 2022-08-15 05:00:03 ['316009316', '316033903']
-4661571458254875855 2022-08-15 05:30:03 ['316009316', '316033903']
-1684920372114538737 2022-08-15 06:00:03 ['316009316', '316033903']
-7171364073050340755 2022-08-15 06:30:03 ['316009316', '316033903']
2598812803747957690 2022-08-15 07:00:03 ['316009316', '316033903']
8816866104682152564 2022-08-15 07:30:03 ['316009316', '316033903']
642686606476463316 2022-08-15 08:00:03 ['316009316', '316033903']
-4076152978603838962 2022-08-15 08:30:03 ['316009316', '316033903']
-7984661426262006031 2022-08-15 09:00:03 ['316009316', '316033903']
-3939472825962773467 2022-08-15 09:30:03 ['316009316', '